In [ ]:
import os
os.chdir('code/DAPPER')

In [ ]:
from dapper.mods.Lorenz96 import dstep_dx, step, x0
import dapper.mods as modelling
import dapper as dpr
import dapper.da_methods as da
import numpy as np

In [ ]:
Nx = 40
obs_noise_level = 1.0
process_noise = 0.0

Force = 8.0
Tplot = 10  # half the burn-in time, maybe used for other stuff too?

p_obs = 0.75

In [ ]:
dt = 0.05
dko = 1  # how many simulation time steps between each observation
Ko = 1000  # number of pseudoobservations produced by our simulation

In [ ]:
x0 = x0(Nx)

Dyn = {
    'M': Nx,
    'model': step,  # time stepping function
    'linear': dstep_dx,  # jacobian of time stepping function
    'noise': process_noise,
}

tseq = modelling.Chronology(dt=dt, dko=dko, Ko=Ko, Tplot=Tplot, BurnIn=2*Tplot)

In [ ]:
mask_array = np.random.rand(Nx, Ko) <= p_obs

def Obs(ko):
    jj = np.flatnonzero(mask_array[:, ko])
    obs = modelling.Operator(**modelining.partial_ID_Obs(Nx, jj), noise=obs_noise_level)
    return obs

In [ ]:
Obs(0)

In [ ]:
X0 = modelling.GaussRV(mu=x0, C=0.001)

In [ ]:
HMM = modelling.HiddenMarkovModel(Dyn, Obs, tseq, X0)

In [ ]:
# #### Run experiment

dpr.set_seed(3000)
xx, yy = HMM.simulate()

In [ ]:
xp = da.EnKF('Sqrt', N=20, infl=1.02, rot=True)
xp.assimilate(HMM, xx, yy, liveplots=True)

In [ ]:
# print(xp.stats)  # ⇒ long printout
xp.stats.average_in_time()
# print(xp.avrgs)  # ⇒ long printout
print(xp.avrgs.tabulate(["rmse.a", "rmse.slow.a", "rmse.fast.a"]))